In [1]:
##Importing all important libraries
import pandas as pd
import numpy as np
import math
import regex as re

import plotly.io as pio
import plotly.graph_objects as go
pio.renderers.default = 'notebook_connected'

### Data Munging Process

In [2]:
##Reading out data set for olympic historical data
event_data=pd.read_csv('../data/athlete_events.csv')
event_data.head()

,ID,Name,Sex,Age,Height,Weight,Team,NOC,Games,Year,Season,City,Sport,Event,Medal
0,1,A Dijiang,M,24.0,180.0,80.0,China,CHN,1992 Summer,1992,Summer,Barcelona,Basketball,Basketball Men's Basketball,NaN
1,2,A Lamusi,M,23.0,170.0,60.0,China,CHN,2012 Summer,2012,Summer,London,Judo,Judo Men's Extra-Lightweight,NaN
2,3,Gunnar Nielsen Aaby,M,24.0,NaN,NaN,Denmark,DEN,1920 Summer,1920,Summer,Antwerpen,Football,Football Men's Football,NaN
3,4,Edgar Lindenau Aabye,M,34.0,NaN,NaN,Denmark/Sweden,DEN,1900 Summer,1900,Summer,Paris,Tug-Of-War,Tug-Of-War Men's Tug-Of-War,Gold
4,5,Christine Jacoba Aaftink,F,21.0,185.0,82.0,Netherlands,NED,1988 Winter,1988,Winter,Calgary,Speed Skating,Speed Skating Women's 500 metres,NaN


In [3]:
##Import data of all the medals won by a country
medal_country=pd.read_csv('../data/medal_by_country.csv')

In [4]:
##Filtering out only the 10 top performing countries 
medal_country=medal_country.loc[0:9]
medal_country


,country,totGold,totSilv,totBrz,totAll,pop2022
0,United States,1180,959,841,2980,334805.269
1,United Kingdom,296,320,332,948,68497.907
2,Germany,293,293,306,892,83883.596
3,France,258,289,327,874,65584.518
4,Italy,257,224,261,742,60262.770
5,China,275,227,194,696,1448471.400
6,Sweden,205,222,234,661,10218.971
7,Australia,169,178,215,562,26068.792
8,Japan,183,172,200,555,125584.838
9,Russia,196,164,187,547,145805.947


In [5]:
##Keeping only the relevant columns 
medal_country=medal_country[['country','totGold','totSilv','totBrz']]
medal_country.head()

,country,totGold,totSilv,totBrz
0,United States,1180,959,841
1,United Kingdom,296,320,332
2,Germany,293,293,306
3,France,258,289,327
4,Italy,257,224,261


In [6]:
##Giving more meaningful names to the columns
medal_country.rename({"totGold":"Gold","totSilv":"Silver","totBrz":"Bronze"},axis=1,inplace=True)

/Users/Tahera/opt/anaconda3/envs/anly503/lib/python3.9/site-packages/pandas/core/frame.py:5039: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [7]:
##Tidying our dataset to prepare for the visualisation
medal_country = pd.melt(medal_country, id_vars='country', value_vars=['Gold', 'Silver', 'Bronze'])

In [8]:
#Loading a dataset to get the continent of each country 
country_continent=pd.read_csv("../data/countries_continent.csv")
country_continent.rename({"Country":"country"},axis=1,inplace=True)
country_continent.head()

,Continent,country
0,Africa,Algeria
1,Africa,Angola
2,Africa,Benin
3,Africa,Botswana
4,Africa,Burkina Faso


In [9]:
##Merging the contients to the medals data 
joining_data = pd.merge(medal_country, country_continent,  on='country')
joining_data=joining_data.drop_duplicates()

In [10]:
##Adding colors of the olympic rings to the dataset
conditions = [
    (joining_data['Continent'] =='North America'), 
    (joining_data["Continent"]=='South America'),
    (joining_data['Continent'] =='Oceania'),
    (joining_data['Continent'] =='Asia'),
    (joining_data['Continent'] =='Europe'),
    (joining_data['Continent'] =='Africa')
]
values = ['#ff0000','#ff0000', '#179a13', '#ffce01', '#3e76ec','#000000']
joining_data['colors'] = np.select(conditions, values)

In [11]:
##In order to plot a Sankey plot, creating numeric values for our labels, these will be the source 
joining_data=joining_data.reset_index()
joining_data['source']=joining_data['index']//3
joining_data.head()

,index,country,variable,value,Continent,colors,source
0,0,United States,Gold,1180,North America,#ff0000,0
1,1,United States,Silver,959,North America,#ff0000,0
2,2,United States,Bronze,841,North America,#ff0000,0
3,3,United Kingdom,Gold,296,Europe,#3e76ec,1
4,4,United Kingdom,Silver,320,Europe,#3e76ec,1


In [12]:
##Providing the numeric labels to our medals that will be the target of the Sankey Plot
conditions2 = [
    (joining_data['variable'] =='Gold'),
    (joining_data['variable'] =='Silver'),
    (joining_data['variable'] =='Bronze')
]
values2 = [10,11,12]
joining_data['target']=np.select(conditions2, values2)

## Sankey Plot

In [13]:
##Creating the different labels of the nodes
labels_list=['United States','United Kingdom','Germany','France','Italy','China','Sweden','Australia','Japan','Russia']
labels_list.extend(("Gold", "Silver", "Bronze"))
##Creating the colors for the nodes
color_list=['#ff0000','#3e76ec','#3e76ec','#3e76ec','#3e76ec','#ffce01','#3e76ec','#179a13','#ffce01','#ffce01']
color_list.extend(("#FFD700", "#C0C0C0", "#CD7F32"))

In [14]:
##Creating the nodes
NODES = dict(label=labels_list,color = color_list,)

#Creating the links between the nodes
LINKS = dict(source = list(joining_data.source), # The origin or the source nodes of the link
target =list(joining_data.target), # The destination or the target nodes of the link
value =  list(joining_data.value), # The width (quantity) of the links
color = 10*["#FFD700", "#C0C0C0", "#CD7F32"],)        


In [15]:
#Developing the plot
data = go.Sankey(node = NODES, link = LINKS)
fig = go.Figure(data)

In [16]:
#Adding our logo for visual encoding
fig=fig.update_layout(
    autosize=False,
    height=500,
    width=900,
    
    
    title=("Top 10 Performing Countries <br>" +
           "<i>1896-2020</i>"),
    title_font_family="Arial",
    title_font_color='black',
    paper_bgcolor='#E7E6E1',
    plot_bgcolor='#E7E6E1')


In [17]:
fig

In [18]:
fig.write_html("../plots/top_performing_countries_ta.html")
